In [6]:
import pandas as pd
import glob
import os
import numpy as np
from featureranker.utils import *
from featureranker.plots import *
from featureranker.rankers import *
from tqdm.auto import tqdm

xlsx_path = './betting_odds/'
txt_path = './game_data/'
paths = [path for path in glob.glob(xlsx_path + '*.xlsx')]
paths_txt = [path_txt for path_txt in glob.glob(txt_path + '*.txt')]
for path in paths:
    xlsx_dataframes = pd.read_excel(path)
    xlsx_dataframes.columns = xlsx_dataframes.columns.str.replace('[^a-zA-Z0-9]', '')
    view_data(xlsx_dataframes)


txt_dataframes = []
for path_txt in paths_txt:
    df = pd.read_csv(path_txt, delimiter = ",", quotechar='"', quoting=1, header = None)
    df.columns = df.columns.astype(str).str.replace('[^a-zA-Z0-9]', '')
    txt_dataframes.append(df)
    view_data(df)
# txt_dataframes = txt_dataframes.replace({',': '', '\"': ''}, regex=True)

# If you want to view the file in Excel, uncomment the following line
# os.startfile(xlsx_files[0])


There are no NaN values in the dataset
The column 9th has 0.1% NaN values.
There are no NaN values in the dataset
There are no NaN values in the dataset
The column 9th has 0.1% NaN values.
There are no NaN values in the dataset
The column 9th has 0.1% NaN values.
The column Pitcher has 0.6% NaN values.
The column 9th has 0.1% NaN values.
The column Pitcher has 0.2% NaN values.
The column 8th has 0.1% NaN values.
The column 9th has 0.1% NaN values.
There are no NaN values in the dataset
There are no NaN values in the dataset
There are no NaN values in the dataset
The column 13 has 99.9% NaN values.
The column 14 has 100.0% NaN values.
The column 15 has 99.9% NaN values.
The column 85 has 100.0% NaN values.
The column 87 has 100.0% NaN values.
The column 97 has 50.5% NaN values.
The column 99 has 5.1% NaN values.
The column 159 has 99.5% NaN values.
The column 13 has 100.0% NaN values.
The column 14 has 100.0% NaN values.
The column 15 has 100.0% NaN values.
The column 81 has 0.3% NaN va

In [7]:
def process_dataframes(xlsx_dataframes):
    merged_df = pd.concat([df for df in xlsx_dataframes])
    home_teams_df = merged_df[merged_df['VH'] == 'H']
    visiting_teams_df = merged_df[merged_df['VH'] == 'V']

    selected_columns_df = pd.DataFrame()
    for i in range(len(xlsx_dataframes)):
        year = 2010 + i
        temp_df = pd.DataFrame({
            'date': home_teams_df['Date'].apply(lambda x: str(year) + str(x).zfill(4)).reset_index(drop=True),
            'home_team': home_teams_df['Team'].reset_index(drop=True),
            'visiting_team': visiting_teams_df['Team'].reset_index(drop=True),
            'home_open': home_teams_df['Open'].reset_index(drop=True),
            'visiting_open': visiting_teams_df['Open'].reset_index(drop=True)
        })
        selected_columns_df = pd.concat([selected_columns_df, temp_df])

    return selected_columns_df

print(process_dataframes([xlsx_dataframes]))


          date home_team visiting_team  home_open  visiting_open
0     20100401       CUB           PIT       -205            183
1     20100401       PHI           ATL       -105           -105
2     20100401       SDG           ARI       -210            188
3     20100401       COL           LAD        188           -210
4     20100401       CIN           STL       -115            105
...        ...       ...           ...        ...            ...
2457  20101027       HOU           ATL       -115           -105
2458  20101029       ATL           HOU       -115           -105
2459  20101030       ATL           HOU       -115           -105
2460  20101031       ATL           HOU       -105           -115
2461  20101102       HOU           ATL       -120            100

[2462 rows x 5 columns]


In [8]:
def add_player_names(selected_columns_df, txt_dataframes):
    player_names_df = pd.DataFrame()
    for txt_df in txt_dataframes:
        txt_df['date'] = txt_df.iloc[:, 0].apply(lambda x: pd.to_datetime(str(x).split(',')[0], format='%Y%m%d', errors='coerce'))
        for i in range(len(selected_columns_df)):
            match_date = selected_columns_df.iloc[i]['date']
            match_home_team = selected_columns_df.iloc[i]['home_team']
            match_visiting_team = selected_columns_df.iloc[i]['visiting_team']
            matching_row = txt_df[(txt_df['date'] == match_date) & (txt_df.iloc[:, 6].values == match_home_team) & (txt_df.iloc[:, 3].values == match_visiting_team)]
            if not matching_row.empty:
                selected_columns_df.at[i, 'home_players'] = matching_row.iloc[:, 132:158].values.tolist()
                selected_columns_df.at[i, 'visiting_players'] = matching_row.iloc[:, 105:131].values.tolist()
    return selected_columns_df

final_df = add_player_names(process_dataframes([xlsx_dataframes]), [txt_dataframes])
print(final_df)









AttributeError: 'list' object has no attribute 'iloc'

In [ ]:
print(selected_columns_df)

NameError: name 'selected_columns_df' is not defined

SyntaxError: invalid syntax (37427434.py, line 1)